paligemma2 
    * Good at question answering.
    * But not good at retrieving the data in markdown or any particular format.

In [1]:
!pip install -U transformers
!pip install accelerate
!pip install huggingface_hub
!pip install pillow


In [2]:
from huggingface_hub import login
login(token="")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/.cache/huggingface/token
Login successful


In [27]:
from transformers import (
    PaliGemmaProcessor,
    PaliGemmaForConditionalGeneration,
)
from transformers.image_utils import load_image
import torch

model_id = "google/paligemma2-10b-ft-docci-448"



model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto").eval()
processor = AutoProcessor.from_pretrained(model_id)



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [35]:
from PIL import Image

url = "/home/Screenshot 2025-01-15 092954.png"
# image = load_image(url)
image = Image.open(url).convert("RGB")

# Leaving the prompt blank for pre-trained models
prompt = "<image> Convert all the information in this image into markdown format. Use headings for titles, bullet points for lists, and code blocks for any code or structured data. Ensure no data is missed."
inputs = processor(prompt, image, return_tensors="pt").to("cuda")
output = model.generate(**inputs, max_new_tokens=1000)

input_len = inputs["input_ids"].shape[-1]
print(processor.decode(output[0][input_len:], skip_special_tokens=True))

Dr. Okan Anyawu, Psychiatry
Progress Note: 07/14/2022 at 2:20 pm
Visit Reason: Med Mgmt
Date of Service:
Patient Name: Adrian Adams
Date of Birth: 01/01/1964
Age: 58 Y
Sex: Male
Race: African American
Ethnicity: Not Hispanic or Latino
Marital Status: Widowed
Employment Status: SEM
Location of Care:
Telehealth visit
Patient seen in office
Accompanied by: No One
Summary:
Patient is here for follow-up visit for: medication management
Diagnoses
Major depressive disorder, recurrent, moderate - F33.1, Primary diagnosis Generalized anxiety disorder - F41.1, Primary diagnosis
Alcohol dependence, uncomplicated - F10.20
Bereavement - Z68.4
Subjective:
Patient states:
-Overall feeling mood has improved, but stilling having emotional breakdowns thinking about wife
-Wife birthday was last weekend,"The weekend was rough"
-Currently in therapy, using prescribed sertraline, 50mg/d
-Stopped taking the Paxil since Sunday, I'm trying to wean off of it"
-Appetite is fair, still only eating once a day whic

In [ ]:

import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor

model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_id)



In [20]:
image = Image.open("/home/Screenshot 2025-01-15 092954.png")

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": "Extract all the data from the given image in markdown format. dont missout anything."}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

output = model.generate(**inputs, max_new_tokens=1000)
print(processor.decode(output[0]))

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

<|image|>Extract all the data from the given image in markdown format. dont missout anything.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Patient Information**

* **Name:** Adrian Adams
* **Date of Birth:** 01/01/1964
* **Age:** 58
* **Sex:** Male
* **Race:** African American
* **Ethnicity:** Not Hispanic or Latino
* **Marital Status:** Widowed
* **Employment Status:** SEM

**Location of Care**

* **Telehealth Visit:** Yes
* **Patient Seen in Office:** No
* **Accompanied by:** No One

**Summary**

* **Patient is here for follow-up visit for:** Medication Management
* **Diagnoses:**
	+ Major Depressive Disorder, Recurrent, Moderate - F33.1, Primary Diagnosis
	+ Generalized Anxiety Disorder - F41.1, Primary Diagnosis
	+ Alcohol Dependence, Uncomplicated - F10.20
	+ Bereavement - Z63.4

**Subjective**

* **Patient Reports:**
	+ Overall feeling mood has improved, but still having emotional breakdowns thinking about wi